This notebook aims to discuss different methods of estimating lightning NOx production efficiency (PE).

## Method 1: time series

Because the TROPOMI passes over the Arctic convection for multiple times, we can use that to calculate the PE and lifetime ($\tau$).

$NO_{2_{T1}}$ = $NO_{2_{T0}}$ $e^{-(T_1-T_0)}$ + $\sum_{i=1}^{m} LNO_{2_{t_i}} e^\frac{{-(t_i-T_0)}}{\tau}$

$NO_{2_{T2}}$ = $NO_{2_{T1}}$ $e^{-(T_2-T_1)}$ + $\sum_{j=1}^{n} LNO_{2_{t_j}} e^\frac{{-(t_j-T_0)}}{\tau}$

= $NO_{2_{T0}}$ $e^{-(T_2-T_0)}$ + $\sum_{i=1}^{m} LNO_{2_{t_i}} e^\frac{{-(T_2-T_0+T_1-t_i)}}{\tau}$ + $\sum_{j=1}^{n} LNO_{2_{t_j}} e^\frac{{-(t_j-T_0)}}{\tau}$

where $T_{<num>}$ is the TROPOMI overpass time,
$NO_{2_{T<num>}}$ is the $NO_2$ above cloud in the mask,
$t_{<num>}$ is the lightning occuring time in the mask,
$LNO_{2_{t<num>}}$ is the produced $LNO_2$ by each flash which is called production efficiency (PE),
and $\tau$ is the $NO_2$ lifetime.

If we assume the PE is constant in the period, then there're only two unknown number (PE and $\tau$) for two equations.

In other words, the three consecutive orbits are useful for estimating the mean PE and $\tau$ from $T_0$ to $T_1$, which can be represented by local time or solar zenith angle.

### Pros

- Lifetime and PE can be derived at the same time

- Don't need to remove background NO2

### Cons

- Smaller SZA leads to larger NO2 VCD. So, there are other NO2 production sources and the $\tau$ may be not actually constant.

- assumption: the cloud is still at the similar level, otherwise the above-cloud NO2 will depends on cloud top height. But, maybe when the cloud is higher, there're some clouds are lower ...

- the PE is the LNO2 PE above cloud


## Method 2: cloud slicing

![cloud slicing](./imgs/Choi_2014.png)

### Prons

- the mean mixing ratio between the highest and lowest upper tropospheric cloud can be derived

### Cons

- the regression is poor

- estimatino of lifetime sounds impossible because of the high regression slope uncertainty

- only works several hours after convection initiation when there're clouds of different levels

- we need to subtract the background NO2 first and then apply cloud slicing

- the mean mixing ratio is the mean upper tropospheric LNO2 mixing ratio

- assumption: the sum of NO2 can stand for the integration of NO2 profile --> LNO2 mixing ratio above cloud

- To get the LNO2 or LNOx PE, we need to apply the lifetime ...

- the NO2 mean mixing ratio needs to be converted into NOx mixing ratio and profile. Note that the NO2:NOx is only well known for ~ 13:30 LT.